In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./3.part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.get(f'''{input_.loc[a, 'Bundle']}theme-bundle.main.js''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    break

            # = = = = = = = = = = = = = = =

            auth = resp.text.split('h.getFitmentData')[1].split('auth=')[1].split('"')[0]

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Part Type': input_.loc[a, 'Part Type'],
                                     'Part Number': input_.loc[a, 'Part Number'],
                                     'Title': input_.loc[a, 'Title'],
                                     'Subtitle': input_.loc[a, 'Subtitle'],
                                     'Json_Info': input_.loc[a, 'Json_Info'],
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': input_.loc[a, 'Json_Src'],
                                     'Auth': auth}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Part Type': part_type,
                                     'Part Number': part_number,
                                     'Title': title,
                                     'Subtitle': subtitle,
                                     'Json_Info': json.dumps(dict_info),
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': json.dumps(dict_src),
                                     'Bundle': input_.loc[a, 'Bundle']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b:}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(9):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./4.auth-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./auth_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：8] - 395.43-1156
[剩余数量：1000] - [当前时间：19:45:27]

[状态：ok，尝试次数：21] - 383.43-1111
[剩余数量：999] - [当前时间：19:45:27]

[状态：ok，尝试次数：36] - 376.43-1008
[剩余数量：998] - [当前时间：19:45:28]

[状态：ok，尝试次数：16] - 397.43-1160
[剩余数量：997] - [当前时间：19:45:28]

[状态：ok，尝试次数：35] - 391.43-1119
[剩余数量：996] - [当前时间：19:45:29]

[状态：ok，尝试次数：90] - 380.43-1036
[剩余数量：995] - [当前时间：19:45:29]

[状态：ok，尝试次数：31] - 393.43-1152
[剩余数量：994] - [当前时间：19:45:29]

[状态：ok，尝试次数：7] - 394.43-1153
[剩余数量：993] - [当前时间：19:45:29]

[状态：ok，尝试次数：49] - 390.43-1118
[剩余数量：992] - [当前时间：19:45:30]

[状态：ok，尝试次数：7] - 403.43-1167
[剩余数量：991] - [当前时间：19:45:31]

[状态：ok，尝试次数：9] - 404.43-1169
[剩余数量：990] - [当前时间：19:45:31]

[状态：ok，尝试次数：2] - 405.43-1170
[剩余数量：989] - [当前时间：19:45:31]

[状态：ok，尝试次数：41] - 396.43-1159
[剩余数量：988] - [当前时间：19:45:32]

[状态：ok，尝试次数：9] - 400.43-1164
[剩余数量：987] - [当前时间：19:45:32]

[状态：ok，尝试次数：88] - 386.43-1114
[剩余数量：986] - [当前时间：19:45:33]

[状态：ok，尝试次数：5] - 398.43-1162
[剩余数量：985] - [当前时间：19:45:33]

[状态：ok，尝试次数：4] - 402.43-1166
[剩余数量：984] - [当前时